In [73]:

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By
from selenium import webdriver
from pprint import pprint

# 強制等待 (執行期間休息一下)
from time import sleep
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
# my_service = Service(executable_path="./chromedriver.exe")



In [74]:
# 開啟chrome driver
driver = webdriver.Chrome(
    options = my_options,
#     service = my_service
)
Position_find_list=[]

In [75]:
driver.get("https://www.104.com.tw/jobs/main/")

In [76]:
list_jobs=['數據分析師', '資料科學家', '資料工程師', 'AI工程師']
Position_find_list=[]
#輸入搜尋條件
driver.find_element(By.CSS_SELECTOR,'input.form-control').send_keys('生醫')
elements = driver.find_elements(By.CSS_SELECTOR,'button span.h3')
for element in elements:
    if element.text == '職務類別': element.click()


In [77]:

# 選擇我想要的職務分類
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, "//span[@class='children' and text()='資訊軟體系統類']")
        )
    )
    driver.find_element(By.XPATH, "//span[@class='children' and text()='資訊軟體系統類']").click()
    driver.find_element(By.XPATH, "//span[@class='children' and text()='軟體／工程類人員']").click()
    
except TimeoutException:
    print("元素未在指定时间内变得可点击")
sleep(3)

In [78]:
# 選取list_jobs裡的職位名並勾取
for job in list_jobs:
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, f"//span[@class='children' and text()='{job}']")
            )   
        )
        driver.find_element(By.XPATH, f"//span[@class='children' and text()='{job}']").click()
    except TimeoutException:
        print("Timeout")

sleep(1)
# press confirm
driver.find_element(By.XPATH, "//button[text()='確定']").click()
# press search
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR,"button[type='submit']")
        )
    )
    driver.find_element(By.CSS_SELECTOR,"button[type='submit']").click()
except TimeoutException:
    print("cant find search")

In [79]:
innerHeight = 0
offset = 0
count = 0
limit = 3
while count<limit: 
    # 取得當前螢幕總長度
    offset = driver.execute_script(
        'return document.documentElement.scrollHeight;'
    )
    # 將螢幕拉到底
    # python f-string中{}為佔位符，要使用{}作為字串得用{{}}
    driver.execute_script(
        f'''window.scrollTo({{top:{offset}, behavior:'smooth'}})''' 
    )
    # 因為有時拉過快會出現手動加載按鈕，判斷如果按鈕出現就按下，沒出現則繼續
    try:
        WebDriverWait(driver, 1.5).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'button.js-more-page')
            )
        )
        driver.find_elements(By.CSS_SELECTOR, 'button.js-more-page')[-1].click()
                 
    except Exception as e:
        print(f"still scrolling well. Error code: {type(e).__name__}, count: {count}")
        # 取得當前高度(用以判斷是否到底)
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )
        # 當快到底scroll過短的時候，經常先取得innerHeight才加載完，導致提早退出
        # 於是發現當前高度=總長度的時候先睡三秒，以免加載慢了
        if innerHeight == offset:
            sleep(3)
            # 睡完後再度判斷是否到底，沒到底則計數器不增加並繼續
            innerHeight = driver.execute_script(
                'return document.documentElement.scrollHeight;'
            )
            if innerHeight == offset:
                count+=1
        
    
        
print('done')

still scrolling well. Error code: TimeoutException, count: 0
still scrolling well. Error code: TimeoutException, count: 0
still scrolling well. Error code: TimeoutException, count: 1
still scrolling well. Error code: TimeoutException, count: 2
done


In [80]:
from bs4 import BeautifulSoup as bs
page_source = driver.page_source
# pprint(page_source)
soup = bs(page_source, "lxml") 

In [81]:
# 爬取資料 jobs_name
elements = soup.select('article[data-qa-id="jobSeachResult"]')
elements
i=0
for ele in elements:
    job_info = {
        "Job_title" : ele.get_attribute_list('data-job-name')[0],
        "Corp_name" : ele.get_attribute_list('data-cust-name')[0],
        "Corp_type" : ele.get_attribute_list('data-indcat-desc')[0]
    }
    Position_find_list.append(job_info)
    
    

In [83]:
import json
import os
folderPath = './job_list/'

def saveJson():
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    with open(f"{folderPath}/jobs_find.json", "w", encoding='utf-8') as file:
        file.write( json.dumps(Position_find_list, ensure_ascii=False, indent=4) )
Position_find_list
saveJson()